## HW3: Decision Tree, AdaBoost and Random Forest
In hw3, you need to implement decision tree, adaboost and random forest by using only numpy, then train your implemented model by the provided dataset. TA will use the on-hold test label to evaluate your model performance.

Please note that only **NUMPY** can be used to implement your model, you will get no points by simply calling `sklearn.tree.DecisionTreeClassifier`

## Question 1
Gini Index or Entropy is often used for measuring the “best” splitting of the data. Please compute the Entropy and Gini Index of provided data. Please use the formula from [page 5 of hw3 slides](https://docs.google.com/presentation/d/1kIe_-YZdemRMmr_3xDy-l0OS2EcLgDH7Uan14tlU5KE/edit#slide=id.gd542a5ff75_0_15)

In [23]:
# Copy and paste your implementations right here to check your result
# (Of course you can add your classes not written here)

import numpy as np
np.random.seed(1)

def gini(sequence):
    
    values, counts = np.unique(sequence, return_counts=True)
    probs = counts / counts.sum()
    
    g = 1.0 - (probs**2.0).sum()
    
    return g


def entropy(sequence):
    
    values, counts = np.unique(sequence, return_counts=True)
    probs = counts / counts.sum()
    
    e = -(probs * np.log2(probs)).sum()

    return e

In [24]:
# 1 = class 1,
# 2 = class 2
data = np.array([1,2,1,1,1,1,2,2,1,1,2])

In [25]:
print("Gini of data is ", gini(data))

Gini of data is  0.4628099173553719


In [26]:
print("Entropy of data is ", entropy(data))

Entropy of data is  0.9456603046006401


## Load data
It is a binary classifiation dataset that classify if price is high or not for a cell phone, the label is stored in `price_range` column.

In [27]:
import pandas as pd

train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')
print(train_df.shape)
print(val_df.shape)

train_df.head()

(1200, 21)
(300, 21)


battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  m_dep  \
0           1583     1          2.1         1  11       0          14    0.7   
1            745     1          0.6         1   5       0          35    0.8   
2            832     0          0.7         1   2       1          39    0.7   
3           1175     1          1.3         0   2       0          19    0.3   
4            695     0          0.5         0  18       1          12    0.6   

   mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  talk_time  \
0        148        7  ...        942      1651  1704    17    13          2   
1        102        8  ...         89      1538  2459    14     1         16   
2        103        4  ...        125      1504  1799     5     2         11   
3        164        7  ...        873      1394  1944     9     4          9   
4        196        2  ...       1649      1829  2855    16    13          7   

   three_g  touch_screen  wifi  price_range  
0        1             0     1            1  
1        1             1     0            0  
2        1             0     1            0  
3        1             1     0            0  
4        1             1     1            1  

[5 rows x 21 columns]

## Question 2
Implement the Decision Tree algorithm (CART, Classification and Regression Trees) and trained the model by the given arguments, and print the accuracy score on the validation data. You should implement two arguments for the Decision Tree algorithm
1. **criterion**: The function to measure the quality of a split. Your model should support `gini` for the Gini impurity and `entropy` for the information gain. 
2. **max_depth**: The maximum depth of the tree. If `max_depth=None`, then nodes are expanded until all leaves are pure. `max_depth=1` equals to split data once


In [28]:
class Node():
    
    def __init__(self):
        
        self.d = -1
        self.t = None
        self.left = None
        self.right = None
        
        self.y = None
        
        return


class DecisionTree():
    def __init__(self, criterion='gini', max_depth=None):
        
        self.criterion = criterion
        self.max_depth = max_depth
        self.max_features = None
        
        self.root = None
        self.d_ls = []
        
        return
    
    def _split(self, x_data, y_data, depth):
        
        node = Node()
        values, counts = np.unique(y_data, return_counts=True)
    
        
        if values.shape[0] == 1:
        
            node.y = values[0]
            
            return node
        
        elif self.max_depth is not None:
            if depth == self.max_depth:
                
                node.y = values[counts.argsort()[-1]]
                
                return node
            
        
        impurity_min = float("inf")
        d_best = -1
        t_best = None

        for d in np.sort(np.random.choice(x_data.shape[1], self.max_features, replace=False)):           
            for t in np.unique(x_data[:, d])[:-1]:
                
                impurity = None
                
                if self.criterion == "gini":
                    
                    y_left = y_data[x_data[:, d]<=t]
                    impurity_left = gini(y_left)
                    y_right = y_data[x_data[:, d]>t]
                    impurity_right = gini(y_right)
                    
                    impurity = y_left.shape[0]/y_data.shape[0]*impurity_left + y_right.shape[0]/y_data.shape[0]*impurity_right

                elif self.criterion == "entropy":
                    
                    y_left = y_data[x_data[:, d]<=t]
                    impurity_left = entropy(y_left)
                    y_right = y_data[x_data[:, d]>t]
                    impurity_right = entropy(y_right)
                    
                    impurity = y_left.shape[0]/y_data.shape[0]*impurity_left + y_right.shape[0]/y_data.shape[0]*impurity_right

                if impurity < impurity_min:

                    impurity_min = impurity
                    d_best = d
                    t_best = t
        
        if d_best == -1:
            
            node.y = values[counts.argsort()[-1]]
            
            return node
        
        
        node.d = d_best
        node.t = t_best
        node.left = self._split(x_data[x_data[:, node.d]<=node.t], y_data[x_data[:, node.d]<=node.t], depth+1)
        node.right = self._split(x_data[x_data[:, node.d]>node.t], y_data[x_data[:, node.d]>node.t], depth+1)
        
        self.d_ls.append(node.d)
        
        return node

    def fit(self, x_data, y_data):
        
        if self.max_features is None:
            self.max_features = x_data.shape[1]
            
        self.root = self._split(x_data, y_data, 0)
        
        return

    def predict(self, x_data):
        
        y_predicts = []
        for x in x_data:
            
            node = self.root
            while(node.y is None):
                if x[node.d] <= node.t:
                    node = node.left
                else:
                    node = node.right
            
            y_predicts.append(node.y)
            
        y_predicts = np.array(y_predicts)
        
        return y_predicts


### Question 2.1
Using `criterion=gini`, showing the accuracy score of validation data by `max_depth=3` and `max_depth=10`, respectively.


In [29]:
clf_depth3 = DecisionTree(criterion='gini', max_depth=3)
clf_depth10 = DecisionTree(criterion='gini', max_depth=10)

x_train = train_df.to_numpy()[:, :-1]
y_train = train_df.to_numpy()[:, -1]
x_val = val_df.to_numpy()[:, :-1]
y_val = val_df.to_numpy()[:, -1]

clf_depth3.fit(x_train, y_train)
y_predicts = clf_depth3.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

clf_depth10.fit(x_train, y_train)
y_predicts = clf_depth10.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

0.9166666666666666
0.9366666666666666


### Question 2.2
Using `max_depth=3`, showing the accuracy score of validation data by `criterion=gini` and `criterion=entropy`, respectively.


In [30]:
clf_gini = DecisionTree(criterion='gini', max_depth=3)
clf_entropy = DecisionTree(criterion='entropy', max_depth=3)

clf_gini.fit(x_train, y_train)
y_predicts = clf_gini.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

clf_entropy.fit(x_train, y_train)
y_predicts = clf_entropy.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

0.9166666666666666
0.93


- Note: Your decisition tree scores should over **0.9**. It may suffer from overfitting, if so, you can tune the hyperparameter such as `max_depth`
- Note: You should get the same results when re-building the model with the same arguments,  no need to prune the trees
- Hint: You can use the recursive method to build the nodes


## Question 3
Plot the [feature importance](https://sefiks.com/2020/04/06/feature-importance-in-decision-trees/) of your Decision Tree model. You can get the feature importance by counting the feature used for splitting data.

- You can simply plot the **counts of feature used** for building tree without normalize the importance. Take the figure below as example, outlook feature has been used for splitting for almost 50 times. Therefore, it has the largest importance

![image](https://i2.wp.com/sefiks.com/wp-content/uploads/2020/04/c45-fi-results.jpg?w=481&ssl=1)

In [31]:
values, counts = np.unique(np.array(clf_depth10.d_ls), return_counts=True)
features = train_df.columns.to_numpy()[values]
features = features[counts.argsort()[::-1]]
counts = counts[counts.argsort()[::-1]]

df_fig = pd.DataFrame({"feature": features, "number": counts})
df_fig.set_index("feature", inplace=True)
fig = df_fig.plot(y="number", kind="bar", legend=False, figsize=(10, 10), fontsize=20, rot=45)

## Question 4
implement the AdaBooest algorithm by using the CART you just implemented from question 2 as base learner. You should implement one arguments for the AdaBooest.
1. **n_estimators**: The maximum number of estimators at which boosting is terminated

In [32]:
class AdaBoost():
    def __init__(self, n_estimators):
        
        self.n_estimators = n_estimators
        
        self.alpha_ls = []
        self.h_ls = []
        
        return

    def fit(self, x_data, y_data):
        
        y_data = 2*y_data - 1
        weights = np.ones(x_data.shape[0]) / x_data.shape[0]
        
        for i in range(self.n_estimators):
            
            indices = np.random.choice(np.arange(x_data.shape[0]), x_data.shape[0], p=weights)
            h = DecisionTree("gini", 1)
            h.fit(x_data[indices], y_data[indices])
            y_predicts = h.predict(x_data) 
            epsilon = weights[y_predicts!=y_data].sum()
            
            alpha = 0.5 * np.log((1-epsilon)/epsilon)
            self.alpha_ls.append(alpha)
            self.h_ls.append(h)
            
            weights = weights * np.exp(-alpha*y_predicts*y_data)
            weights = weights / weights.sum()
            
        return

    def predict(self, x_data):
        
        y_predicts = np.zeros(x_data.shape[0])
        for alpha, h in zip(self.alpha_ls, self.h_ls):
            y_predicts = y_predicts + alpha*h.predict(x_data)
        
        y_predicts = (y_predicts >= 0.0).astype("int")
        
        return y_predicts

### Question 4.1
Show the accuracy score of validation data by `n_estimators=10` and `n_estimators=100`, respectively.


In [33]:
clf_10 = AdaBoost(10)
clf_100 = AdaBoost(100)

clf_10.fit(x_train, y_train)
y_predicts = clf_10.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

clf_100.fit(x_train, y_train)
y_predicts = clf_100.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

0.9466666666666667
0.9733333333333334


## Question 5
implement the Random Forest algorithm by using the CART you just implemented from question 2. You should implement three arguments for the Random Forest.

1. **n_estimators**: The number of trees in the forest. 
2. **max_features**: The number of random select features to consider when looking for the best split
3. **bootstrap**: Whether bootstrap samples are used when building tree


In [34]:
class RandomForest():
    def __init__(self, n_estimators, max_features, boostrap=True, criterion='gini', max_depth=None):
        
        self.n_estimators = n_estimators
        self.max_features = int(max_features)
        self.boostrap = boostrap
        self.criterion = criterion
        self.max_depth = max_depth
        
        self.decisiontree_ls = []
        
        return

    def fit(self, x_data, y_data):
        
        for i in range(self.n_estimators):
            
            if self.boostrap:
                indices = np.random.choice(x_data.shape[0], x_data.shape[0])
            else:
                indices = np.arange(x_data.shape[0])
            
            decisiontree = DecisionTree(self.criterion, self.max_depth)
            decisiontree.max_features = self.max_features
            decisiontree.fit(x_data[indices], y_data[indices])
            
            self.decisiontree_ls.append(decisiontree)
            
        return

    def predict(self, x_data):
        
        y_predicts = np.zeros(x_data.shape[0])
        for decisiontree in self.decisiontree_ls:
            y_predicts = y_predicts + decisiontree.predict(x_data)
        
        y_predicts = y_predicts / len(self.decisiontree_ls)
        y_predicts = (y_predicts >= 0.5).astype("int")
        
        return y_predicts

### Question 5.1
Using `criterion=gini`, `max_depth=None`, `max_features=sqrt(n_features)`, showing the accuracy score of validation data by `n_estimators=10` and `n_estimators=100`, respectively.


In [35]:
clf_10tree = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))
clf_100tree = RandomForest(n_estimators=100, max_features=np.sqrt(x_train.shape[1]))

In [36]:
clf_10tree.fit(x_train, y_train)
y_predicts = clf_10tree.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

clf_100tree.fit(x_train, y_train)
y_predicts = clf_100tree.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

0.9266666666666666
0.9433333333333334


### Question 5.2
Using `criterion=gini`, `max_depth=None`, `n_estimators=10`, showing the accuracy score of validation data by `max_features=sqrt(n_features)` and `max_features=n_features`, respectively.


In [37]:
clf_random_features = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))
clf_all_features = RandomForest(n_estimators=10, max_features=x_train.shape[1])

- Note: Use majority votes to get the final prediction, you may get slightly different results when re-building the random forest model

In [38]:
clf_random_features.fit(x_train, y_train)
y_predicts = clf_random_features.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

clf_all_features.fit(x_train, y_train)
y_predicts = clf_all_features.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

0.9133333333333333
0.9566666666666667


### Question 6. Train and tune your model on a real-world dataset
Try you best to get higher accuracy score of your model. After parameter tuning, you can train your model on the full dataset (train + val).
- Feature engineering
- Hyperparameter tuning
- Implement any other ensemble methods, such as gradient boosting. Please note that you **can not** call any package. Also, only ensemble method can be used. Neural network method is not allowed to used.

In [39]:
def train_your_model(data):
    ## Define your model and training 
    
    clf_101 = AdaBoost(101)
    clf_101.fit(data.to_numpy()[:, :-1], data.to_numpy()[:, -1])
    
    return clf_101

In [40]:
my_model = train_your_model(train_df)
y_predicts = my_model.predict(x_val)
print((y_predicts==y_val).sum() / y_val.shape[0])

0.97


In [42]:
my_model = train_your_model(pd.concat([train_df, val_df]))
# y_predicts = my_model.predict(x_val)
# print((y_predicts==y_val).sum() / y_val.shape[0])
x_test = pd.read_csv('x_test.csv').values
y_pred = my_model.predict(x_test)

IndexError: index 13 is out of bounds for axis 0 with size 7

In [43]:
assert y_pred.shape == (500, )

NameError: name 'y_pred' is not defined

## Supplementary
If you have trouble to implement this homework, TA strongly recommend watching [this video](https://www.youtube.com/watch?v=LDRbO9a6XPU), which explains Decision Tree model clearly. But don't copy code from any resources, try to finish this homework by yourself! 

### DO NOT MODIFY CODE BELOW

In [44]:
import pandas as pd
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('y_test.csv')['price_range'].values

print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

FileNotFoundError: [Errno 2] No such file or directory: 'y_test.csv'

In [45]:
def discrete_checker(score, thres, clf, name, x_train, y_train, x_test, y_test):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    if accuracy_score(y_test, y_pred) - thres >= 0:
        return score
    else:
        print(f"{name} failed")
        return 0


def patient_checker(score, thres, CLS, kwargs, name,
                    x_train, y_train, x_test, y_test, patient=10):
    while patient > 0:
        patient -= 1
        clf = CLS(**kwargs)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        if accuracy_score(y_test, y_pred) - thres >= 0:
            return score
    print(f"{name} failed")
    print("Considering the randomness, we will check it manually")
    return 0


def load_dataset():
    file_url = "http://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv"
    df = pd.read_csv(
        file_url,
        names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
               "Viscera weight", "Shell weight", "Age"]
    )

    df['Target'] = (df["Age"] > 15).astype(int)
    df = df.drop(labels=["Age"], axis="columns")

    train_idx = range(0, len(df), 10)
    test_idx = range(1, len(df), 20)

    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

    x_train = train_df.drop(labels=["Target"], axis="columns")
    feature_names = x_train.columns.values
    x_train = x_train.values
    y_train = train_df['Target'].values

    x_test = test_df.drop(labels=["Target"], axis="columns")
    x_test = x_test.values
    y_test = test_df['Target'].values
    return x_train, y_train, x_test, y_test, feature_names


score = 0

data = np.array([1, 2])
if abs(gini(data) - 0.5) < 1e-4:
    score += 2.5
else:
    print("gini test failed")

if abs(entropy(data) - 1) < 1e-4:
    score += 2.5
else:
    print("entropy test failed")

x_train, y_train, x_test, y_test, feature_names = load_dataset()

score += discrete_checker(5, 0.9337,
                          DecisionTree(criterion='gini', max_depth=3),
                          "DecisionTree(criterion='gini', max_depth=3)",
                          x_train, y_train, x_test, y_test
                          )

score += discrete_checker(2.5, 0.9036,
                          DecisionTree(criterion='gini', max_depth=10),
                          "DecisionTree(criterion='gini', max_depth=10)",
                          x_train, y_train, x_test, y_test
                          )

score += discrete_checker(2.5, 0.9096,
                          DecisionTree(criterion='entropy', max_depth=3),
                          "DecisionTree(criterion='entropy', max_depth=3)",
                          x_train, y_train, x_test, y_test
                          )

print("*** We will check your result for Question 3 manually *** (5 points)")

score += patient_checker(
    7.5, 0.91, AdaBoost, {"n_estimators": 10},
    "AdaBoost(n_estimators=10)",
    x_train, y_train, x_test, y_test
)

score += patient_checker(
    7.5, 0.87, AdaBoost, {"n_estimators": 100},
    "AdaBoost(n_estimators=100)",
    x_train, y_train, x_test, y_test
)

score += patient_checker(
    5, 0.91, RandomForest,
    {"n_estimators": 10, "max_features": np.sqrt(x_train.shape[1])},
    "RandomForest(n_estimators=10, max_features=sqrt(n_features))",
    x_train, y_train, x_test, y_test
)

score += patient_checker(
    5, 0.91, RandomForest,
    {"n_estimators": 100, "max_features": np.sqrt(x_train.shape[1])},
    "RandomForest(n_estimators=100, max_features=sqrt(n_features))",
    x_train, y_train, x_test, y_test
)

score += patient_checker(
    5, 0.92, RandomForest,
    {"n_estimators": 10, "max_features": x_train.shape[1]},
    "RandomForest(n_estimators=10, max_features=n_features)",
    x_train, y_train, x_test, y_test
)

print("*** We will check your result for Question 6 manually *** (20 points)")
print("Approximate score range:", score, "~", score + 25)
print("*** This score is only for reference ***")


*** We will check your result for Question 3 manually *** (5 points)
*** We will check your result for Question 6 manually *** (20 points)
Approximate score range: 45.0 ~ 70.0
*** This score is only for reference ***
